### Europian Option Pricing using Black-Scholes Model

In [5]:
## import libraries
import numpy as np
import pandas as pd
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date

Black-Scholes Model give the option price by:

**𝐂=𝑆𝑁(𝑑1)−𝐾𝑒xp(−𝑟𝑡)𝑁(𝑑2)**

**P=𝐾𝑒xp(−𝑟𝑡)𝑁(-𝑑2)-𝑆𝑁(-𝑑1)**


- 𝑑1 = (ln(𝑆/𝐾)+t(𝑟+(sigma^2)/2) ) / sigma√𝑡
- 𝑑2 = 𝑑1 − sigma√𝑡 
where:

C=Call option price
P = Put Option price

S=Current stock (or other underlying) price
K=Strike price

r=Risk-free interest rate
t=Time to maturity
N=Standard normal cumulative distribution function
sigma = Underlying asset volatility

**Defining the functions for the Black-Scholes Formula**

In [15]:
def d1(S,K,T,r,sigma):
    return (log(S/K)+(r+sigma**2/2.)*T)/sigma*sqrt(T)
def d2(S,K,T,r,sigma):
    return d1 (S,K,T,r,sigma)-sigma*sqrt(T)

In [16]:
## Black-Scholes Option Pricing

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

**Option Greeks**

Below is the list of Option Greeks that I have computed along with their brief description:

1. Delta: The change in the option price with 1 unit change in underlying asset price. In other words, the price sensitivity of the option is relative to the underlying asset
2. Gamma: The rate of change of option's delta with respect to underlying asset price. This is called second-order (second-derivative) price sensitivity.
3. Vega: The rate of change in the option price with respect to the volatility in the underlying asset.
4. Rho: The rate of change in of the option price respected to the risk-free rate.
5. Theta: The rate of change in the option price with respect to the time of expiry. This is also known as Option's time decay    

In [17]:
## Delta
def call_delta(S,K,T,r,sigma):
    return norm.cdf(d1(S,K,T,r,sigma))

def put_delta(S,K,T,r,sigma):
    return -norm.cdf(-d1(S,K,T,r,sigma))


## Gamma
def call_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))

def put_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))


## Vega
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))

def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))


## Theta
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))

def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))


## Rho
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))

def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))

**Taking input from the user**

In [20]:
## Inputs from the User

S = input("Stock price?");
while True:
    try:
        S = float(S)
        break
    except:
        print("Please enter a Number")
        S = input("Stock price?")

        
K = input("Strike price?");
while True:
    try:
        K = float(K)
        break
    except:
        print("Please enter a Number")
        K = input("Strike price?")


while True:
    time_to_expiry = input("Option Expiration Date? (yyyy-mm-dd) ")
    try:
        time_to_expiry = datetime.strptime(time_to_expiry, "%Y-%m-%d")
    except ValueError as e:
        print("error: %s\nTry again." % (e,))
    else:
        break
T = (time_to_expiry - datetime.utcnow()).days / 365


r = input("Risk-free rate for this period(%)?");
while True:
    try:
        r = float(r)
        break
    except:
        print("Please enter a Number.")
        r = input("Risk-free rate for this period(%)")
        
## input the volatility and check if it is a number.
sigma = input("Volatility(%)?"); 
while True:
    try:
        sigma = float(sigma)
        if sigma > 100 or sigma < 0:
            print ( "Enter a value between [0,100]")
            sigma = input("Volatility(%)?")
        break
    except:
        print ( "Enter a value between [0,100]")
        sigma = input("Volatility(%)?")


Stock price?100
Strike price?99
Option Expiration Date? (yyyy-mm-dd) 2020-12-31
Risk-free rate for this period(%)?2
Volatility(%)?5


In [21]:
data = {'Field': ['S', 'K', 'T', 'r', 'sigma'],
        'Value': [S, K, T , r , sigma]}
df1 = pd.DataFrame(data, columns=['Field', 'Value'], 
                   index=['Stock price', 'Strike price', 'Time to maturity', 'Risk-free rate', 'Stock Volatility'])
df1

,Field,Value
Stock price,S,100.000000
Strike price,K,99.000000
Time to maturity,T,0.084932
Risk-free rate,r,2.000000
Stock Volatility,sigma,5.000000


In [22]:
r = r/100; sigma = sigma/100;
metrics = {'Call' : [bs_call(S,K,T,r,sigma), call_delta(S,K,T,r,sigma), call_gamma(S,K,T,r,sigma),call_vega(S,K,T,r,sigma), call_rho(S,K,T,r,sigma), call_theta(S,K,T,r,sigma)],
                    'Put' : [bs_put(S,K,T,r,sigma), put_delta(S,K,T,r,sigma), put_gamma(S,K,T,r,sigma),put_vega(S,K,T,r,sigma), put_rho(S,K,T,r,sigma), put_theta(S,K,T,r,sigma)]}
df2 = pd.DataFrame(metrics, columns=['Call','Put'], index=['Price', 'delta', 'gamma','vega','rho','theta'])
df2

,Call,Put
Price,1.189611,0.021589
delta,0.527545,-0.472455
gamma,0.273130,0.273130
vega,0.115987,0.115987
rho,0.043795,-0.040145
theta,-0.044454,-0.024688


**Conclusion:**
Based on the user defined input, this above dataframe displays the output for Call and Put Option Price along with the Option Greeks 

I will further add the Implied Volatility calculation in the next part of this Notebook